# Table of Contents

- Build a list of snowpack
- Build a DataFrame of snowpack

**How to run a sensitivity study on snowpack properties ?**

SMRT is able to run the model for a sensor that have a list of frequencies, angles or polarization automatically. However, for snowpack properties setting for instance the corr_length in a layer to a list (or array) does not work (yet). It is necessary to do a little more programming, but it is not so complex.

First import the various module we always need:

In [ ]:
import numpy as np
import pandas as pd

import smrt
import matplotlib.pyplot as plt
%matplotlib widget

Build a list of snowpack
==========================

We want to test the sensitivity of TB's to the corr_length. We first build a list of snowpack with different corr_length

In [ ]:
# prepare the snowpack
density = 300.0
corr_length = np.arange(0, 0.5, 0.01) * 1e-3  # from 0 to 0.5mm

# the naive way: without list comprehension

snowpack = list()
for x in corr_length:
    sp = smrt.make_snowpack([1000], "exponential", density=[density], temperature=[265], corr_length=x)
    snowpack.append(sp)

A better way is to use list comprehension. To achieve the same as these latter 4 last lines:

In [ ]:
# the one-liner one: with list comprehension (start reading by the end)
snowpack = [smrt.make_snowpack([1000], "exponential", density=[density], temperature=[265], corr_length=x) for x in corr_length]

Run the model
================

Let's first define a model and a sensor. 

In [ ]:
model = smrt.make_model("iba", "dort")
sensor = smrt.sensor_list.passive(37e9, 55)

Running the model with a list of snowpack is possible and even the recommended way. It is more efficient (with parallel_computation=True) and the `results` object contains all the simulation results.

In [ ]:
results = model.run(sensor, snowpack, snowpack_dimension=('corr_length', corr_length))
print(type(results))  # look results is a Result
print()
print(results.coords) # look, we have several coordinates, one is call corr_legn

Now we can call TbV() directly because we have a Result instance

In [ ]:
plt.figure()
plt.plot(corr_length, results.TbV())

Build a DataFrame of snowpack
==============================

Using pandas with SMRT provides even easier way to run many simulations with complex dependence or multi-variation sensitivity anylisis

In [ ]:
# start with a DataFrame with the main parameters
# here we consider only one parameter
snowpack_df = pd.DataFrame({'corr_length': np.arange(0, 0.5, 0.01) * 1e-3})  # from 0 to 0.5mm

# build the snowpacks and create a column in the dataframe.


snowpack_df['snowpack'] = [smrt.make_snowpack([1000], "exponential",
                                     density=[density], 
                                     temperature=[265],
                                     corr_length=row['corr_length']) for i, row in snowpack_df.iterrows()]

snowpack_df.head()

In [ ]:
# run the model and convert the results to a dataframe
sensor = smrt.sensor_list.amsre(['19', '37'])

results = model.run(sensor, snowpack_df).to_dataframe()
print(type(results))  # look results is a Result
print()
results.set_index('corr_length')  # look, we have several coordinates, one is call corr_legn